In [1]:
import pandas as pd

In [2]:
fc = pd.read_csv('../../etl_data/fail_categories.csv')

In [3]:
keep_cols = ['Date', 'Total Failed', 'Tested Batches', 'Failed Batches', 'Failure Reason']
fc = fc[keep_cols]
fc

,Date,Total Failed,Tested Batches,Failed Batches,Failure Reason
0,2018-11-13,3375,20797,2043,Label Claims
1,2018-11-13,3375,20797,685,Pesticides
2,2018-11-13,3375,20797,380,Microbial Impurities
3,2018-11-13,3375,20797,171,Residual Solvents
4,2018-11-13,3375,20797,35,Homogeneity
...,...,...,...,...,...
736,2019-04-15,4699,44017,8,Water Activity
737,2019-04-15,4699,44017,26,Cannabinoids
738,2019-04-15,4699,44017,1,Mycotoxins
739,2019-04-15,4699,44017,2,Injurious to Human Health


In [4]:
fc_pivot = fc.pivot_table(index='Date', columns='Failure Reason')
fc_pivot.fillna(value=0, inplace=True) # NaNs represent 0s in reality
fc_pivot = fc_pivot.astype('int')
fc_pivot.sort_index(inplace=True)
fc_pivot['Date'] = pd.to_datetime(fc_pivot.index) # Need a 'Date' column for .diff() to calculate num_days
fc_pivot

Failed Batches                                            \
Failure Reason   Cannabinoids Foreign Material Heavy Metals Homogeneity   
Date                                                                      
2018-10-09                  0               10            0          35   
2018-10-15                  0               10            0          35   
2018-10-22                  0               10            0          35   
2018-10-29                  0               10            0          35   
2018-11-05                  0               10            0          35   
...                       ...              ...          ...         ...   
2020-01-13                 74               25          531          36   
2020-01-20                 77               26          541          36   
2020-01-27                 79               26          552          36   
2020-02-03                 81               27          573          36   
2020-02-10                 81               27          584          36   

                                                                            \
Failure Reason Injurious to Human Health Label Claims Microbial Impurities   
Date                                                                         
2018-10-09                             0         1751                  201   
2018-10-15                             0         1823                  217   
2018-10-22                             0         1867                  224   
2018-10-29                             0         1953                  233   
2018-11-05                             0         1979                  249   
...                                  ...          ...                  ...   
2020-01-13                             2         2379                  910   
2020-01-20                             2         2379                  929   
2020-01-27                             2         2379                  940   
2020-02-03                             2         2379                  955   
2020-02-10                             2         2379                  978   

                                               ...              Total Failed  \
Failure Reason Moisture Mycotoxins Pesticides  ... Injurious to Human Health   
Date                                           ...                             
2018-10-09           40          0        602  ...                         0   
2018-10-15           45          0        632  ...                         0   
2018-10-22           45          0        645  ...                         0   
2018-10-29           45          0        663  ...                         0   
2018-11-05           50          0        672  ...                         0   
...                 ...        ...        ...  ...                       ...   
2020-01-13           71         10       1777  ...                      6223   
2020-01-20           71         10       1790  ...                      6273   
2020-01-27           71         10       1804  ...                      6314   
2020-02-03           71         10       1822  ...                      6377   
2020-02-10           71         10       1833  ...                      6424   

                                                                      \
Failure Reason Label Claims Microbial Impurities Moisture Mycotoxins   
Date                                                                   
2018-10-09             2769                 2769     2769          0   
2018-10-15             2899                 2899     2899          0   
2018-10-22             2971                 2971     2971          0   
2018-10-29             3093                 3093     3093          0   
2018-11-05             3162                 3162     3162          0   
...                     ...                  ...      ...        ...   
2020-01-13             6223                 6223     6223       6223   
2020-01-20             6273   

In [5]:
delta_fc = fc_pivot.diff()
delta_fc = delta_fc.iloc[1:] # Drop first row, which is all NaNs

# Convert to int's 
keep_cols = ['Failed Batches', 'Tested Batches', 'Total Failed']
for col in keep_cols:
    delta_fc[col] = delta_fc[col].astype('int')
delta_fc.dtypes

delta_fc

Failed Batches                                            \
Failure Reason   Cannabinoids Foreign Material Heavy Metals Homogeneity   
Date                                                                      
2018-10-15                  0                0            0           0   
2018-10-22                  0                0            0           0   
2018-10-29                  0                0            0           0   
2018-11-05                  0                0            0           0   
2018-11-13                  0                0            0           0   
...                       ...              ...          ...         ...   
2020-01-13                  1                0            5           0   
2020-01-20                  3                1           10           0   
2020-01-27                  2                0           11           0   
2020-02-03                  2                1           21           0   
2020-02-10                  0                0           11           0   

                                                                            \
Failure Reason Injurious to Human Health Label Claims Microbial Impurities   
Date                                                                         
2018-10-15                             0           72                   16   
2018-10-22                             0           44                    7   
2018-10-29                             0           86                    9   
2018-11-05                             0           26                   16   
2018-11-13                             0           64                  131   
...                                  ...          ...                  ...   
2020-01-13                             0            0                   18   
2020-01-20                             0            0                   19   
2020-01-27                             0            0                   11   
2020-02-03                             0            0                   15   
2020-02-10                             0            0                   23   

                                               ...              Total Failed  \
Failure Reason Moisture Mycotoxins Pesticides  ... Injurious to Human Health   
Date                                           ...                             
2018-10-15            5          0         30  ...                         0   
2018-10-22            0          0         13  ...                         0   
2018-10-29            0          0         18  ...                         0   
2018-11-05            5          0          9  ...                         0   
2018-11-13            1          0         13  ...                         0   
...                 ...        ...        ...  ...                       ...   
2020-01-13            0          2          9  ...                        38   
2020-01-20            0          0         13  ...                        50   
2020-01-27            0          0         14  ...                        41   
2020-02-03            0          0         18  ...                        63   
2020-02-10            0          0         11  ...                        47   

                                                                      \
Failure Reason Label Claims Microbial Impurities Moisture Mycotoxins   
Date                                                                   
2018-10-15              130                  130      130          0   
2018-10-22               72                   72       72          0   
2018-10-29              122                  122      122          0   
2018-11-05               69                   69       69          0   
2018-11-13              213                  213      213          0   
...                     ...                  ...      ...        ...   
2020-01-13               38                   38       38         38   
2020-01-20               50   

In [6]:
delta_fc['Date']

Date
2018-10-15   6 days
2018-10-22   7 days
2018-10-29   7 days
2018-11-05   7 days
2018-11-13   8 days
              ...  
2020-01-13   7 days
2020-01-20   7 days
2020-01-27   7 days
2020-02-03   7 days
2020-02-10   7 days
Name: Date, Length: 62, dtype: timedelta64[ns]

In [7]:
delta_fc['Failed Batches'] / delta_fc['Date']

TypeError: unsupported operand type(s) for /: 'int' and 'NaTType'

In [ ]:
list(delta_fc['Date'])